## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
from datetime import datetime
from scipy import stats  
import os
import pytz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  

%matplotlib inline

### Creating IDX Equities Time Series

In [2]:
# idx_combined = pd.read_csv('/home/nikki/documents/amibroker/idx_exported_combined.csv', parse_dates=[1])

In [2]:
# Location of the IDX Stocks OHLC csv files
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv'
# csv_loc = '/Users/nikki/Documents/idx_exported_csv'
csv_loc = '/home/nikki/documents/amibroker/exported-csv/'

# Backup if the first one doesn't work
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv_gama'

tickers = os.listdir(csv_loc)

for ticker in tickers:
    if len(ticker) != 8:
        print(f'ticker')

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for ticker in tickers:
    data_idx[ticker[:-4]] = pd.read_csv(f'{csv_loc}/{ticker}',
                                        index_col='date',
                                        parse_dates=['date'])

In [4]:
data_idx['ASII'].tail()

,open,high,low,close,volume
date,,,,,
2021-01-25,6650.0,6650.0,6325.0,6475.0,73391696
2021-01-26,6475.0,6575.0,6300.0,6350.0,39158300
2021-01-27,6350.0,6350.0,6150.0,6275.0,56444100
2021-01-28,6150.0,6450.0,6150.0,6275.0,54596400
2021-01-29,6300.0,6375.0,5950.0,6100.0,71886200


### Stocks on the Move


In [5]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [6]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [18]:
# How many (series) candles back for momentum calculation?
momentum_window = 96

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# Eliminated stocks list
eliminated_stocks = pd.DataFrame(columns=['ticker', 'score', 'vola', 'reason'])

# How many (series) candles+1 back for std dev calculation?
vola_window = 24

# How many (series) candles back for EWMA calculation?
ma_period_fast = 32
ma_period_slow = 128

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    momentum_series = timeseries['close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['close']
    vola = volatility(vola_series, vola_window) * 16
    median_volume = timeseries['volume'].rolling(vola_window).median().iloc[-1]
    ma_fast = timeseries['close'].rolling(ma_period_fast).mean().iloc[-1]
    ma_slow = timeseries['close'].rolling(ma_period_slow).mean().iloc[-1]

#     ewma = timeseries['close'].ewm(span=ewma_period).mean().iloc[-1]

    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'umur belum 3 tahun'},
                                                     ignore_index=True)
        continue
    
    # If median volume falls below 100k in the stocks, drop it 
    if median_volume < 100000:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'avg volume di bawah 100k'},
                                                     ignore_index=True)
        continue
        
    # If it has been suspended (daily vol == 0) more than once, drop it
    if timeseries['volume'].iloc[-momentum_window:].tolist().count(0) > 1:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'pernah disuspend lebih dari 1x'},
                                                     ignore_index=True)
        continue
    
    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'median_vol': median_volume,
                                            'ma_fast': ma_fast,
                                            'ma_slow': ma_slow},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

/home/nikki/miniconda3/envs/anaconda/lib/python3.8/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/nikki/miniconda3/envs/anaconda/lib/python3.8/site-packages/numpy/lib/function_base.py:2449: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [19]:
print(f'Ada {len(momentum_table)} saham lolos')
print(f'Ada {len(eliminated_stocks)} saham tereliminasi')

Ada 261 saham lolos
Ada 458 saham tereliminasi


In [20]:
momentum_table.sort_values('score', ascending=False)[:50].to_clipboard()

In [16]:
momentum_table.sort_values('score', ascending=False)[:50]

,ticker,score,vola,inv_vola,ma,median_vol
214,BBHI,2853.426633,1.759296,0.568409,222.203125,3899450.0
163,SAME,2841.954346,0.886679,1.127804,159.335938,105598800.0
189,ANTM,2341.379244,1.020847,0.979579,1117.421875,713320256.0
195,WSKT,2118.031042,0.894024,1.118538,865.609375,216926152.0
152,ASRI,1447.780909,0.596036,1.677751,164.812500,219027848.0
129,BAJA,1301.954478,0.461714,2.165842,87.757812,4321250.0
177,AGII,1205.752872,1.007247,0.992805,681.750000,1844450.0
87,SQMI,1183.548869,0.523294,1.910972,226.179688,8026400.0
29,HRUM,1150.053679,0.838282,1.192916,2027.617188,7383350.0
8,TGRA,1128.547551,1.675885,0.596700,106.593750,64526750.0


In [13]:
eliminated_stocks[eliminated_stocks['reason'] == 'umur belum 3 tahun'].sort_values('score', ascending=False)

,ticker,score,vola,reason
40,BRIS,3815.691907,9.371545e-01,umur belum 3 tahun
82,DMMX,1200.297792,6.199791e-01,umur belum 3 tahun
44,CBMF,601.234993,3.262737e-01,umur belum 3 tahun
171,SAPX,529.212255,3.554760e-01,umur belum 3 tahun
43,INDO,466.787826,4.488161e-01,umur belum 3 tahun
...,...,...,...,...
83,KBAG,-85.764465,1.415350e+00,umur belum 3 tahun
0,ITIC,-90.342123,4.416746e-01,umur belum 3 tahun
172,HRME,-91.511054,5.381667e-09,umur belum 3 tahun
209,ARKA,-92.140816,5.981414e-01,umur belum 3 tahun


In [14]:
eliminated_stocks[eliminated_stocks['reason'] == 'pernah disuspend lebih dari 1x'].sort_values('score', ascending=False)

,ticker,score,vola,reason
163,TIFA,779.395081,0.305628,pernah disuspend lebih dari 1x
261,INCI,272.108060,0.410409,pernah disuspend lebih dari 1x
128,RANC,100.071218,0.348120,pernah disuspend lebih dari 1x
39,RMBA,74.438505,1.595118,pernah disuspend lebih dari 1x
307,RUIS,42.147570,0.923970,pernah disuspend lebih dari 1x
488,BIPP,39.918142,0.606751,pernah disuspend lebih dari 1x
109,WINS,35.294318,0.213005,pernah disuspend lebih dari 1x
60,LPPS,15.956782,0.699902,pernah disuspend lebih dari 1x
218,GWSA,13.444831,0.528288,pernah disuspend lebih dari 1x
34,AISA,8.955749,1.638821,pernah disuspend lebih dari 1x


In [15]:
eliminated_stocks[eliminated_stocks['reason'] == 'avg volume di bawah 100k'].sort_values('score', ascending=False)

,ticker,score,vola,reason
77,ARGO,489.785755,1.123822e+00,avg volume di bawah 100k
159,PTIS,358.443206,9.486231e-01,avg volume di bawah 100k
408,BBHI,323.138029,5.734052e-01,avg volume di bawah 100k
436,MERK,306.264092,4.505298e-01,avg volume di bawah 100k
358,PNSE,165.606997,3.973270e-07,avg volume di bawah 100k
...,...,...,...,...
70,PGLI,-66.517070,1.117693e+00,avg volume di bawah 100k
377,NASA,-68.554597,2.249972e-08,avg volume di bawah 100k
419,TARA,-71.126410,3.709058e-08,avg volume di bawah 100k
373,GLOB,-79.339016,5.216332e-01,avg volume di bawah 100k
